## Univariate LSTM-RNN Sumatera Selatan Percobaan 1

### 1. Deklarasi Pustaka

In [1]:
# pustaka untuk manipulasi data
import pandas as pd
from pandas import concat
from pandas import DataFrame
from pandas import read_csv
from pandas import read_excel
import numpy as np
from numpy import concatenate
from numpy import array

# pustaka untuk waktu komputasi
import time
from datetime import datetime

# Pustaka untuk visualisasi data
import seaborn as sns
from matplotlib import pyplot
from matplotlib import pyplot as plt

# Pustaka untuk visualisasi acf dan pacf
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
        
# Pustaka untuk membuat data latih dan data uji.
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

# Pustaka untuk membuat model prediksi LSTM-RNN
import itertools
import tensorflow as tf
from keras.models import Sequential
from keras.layers import RNN
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from keras.optimizers import Adam, Adamax, RMSprop, SGD

# Early stoping
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

# Pustaka untuk  evaluasi model prediksi
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

### 2. Akuisisi Data

In [2]:
# Set waktu komputasi
start = time.time()

In [3]:
# fix random seed for reproducibility
np.random.seed(1234)

In [4]:
# membaca dataset via csv file
dataset = pd.read_csv("dataset/modis_sumsel_2001_2021_v1.csv", parse_dates=['acq_date'], engine="python")

In [5]:
# menampilkan metadata dataset
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155661 entries, 0 to 155660
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   latitude    155661 non-null  float64       
 1   longitude   155661 non-null  float64       
 2   brightness  155661 non-null  float64       
 3   scan        155661 non-null  float64       
 4   track       155661 non-null  float64       
 5   acq_date    155661 non-null  datetime64[ns]
 6   acq_time    155661 non-null  int64         
 7   satellite   155661 non-null  object        
 8   instrument  155661 non-null  object        
 9   confidence  155661 non-null  int64         
 10  version     155661 non-null  float64       
 11  bright_t31  155661 non-null  float64       
 12  frp         155661 non-null  float64       
 13  daynight    155661 non-null  object        
 14  type        155661 non-null  int64         
dtypes: datetime64[ns](1), float64(8), int64(3), object(

In [6]:
# menampilkan dataset
dataset

latitude  longitude  brightness  scan  track   acq_date  acq_time  \
0        -4.8609   104.1465       320.4   1.0    1.0 2002-10-20       330   
1        -4.9018   104.0954       322.3   1.1    1.0 2006-09-27       635   
2        -4.8926   104.0940       335.7   1.1    1.0 2006-09-27       635   
3        -4.7994   103.8955       331.9   1.2    1.1 2011-06-10       653   
4        -4.7799   103.7999       319.2   1.1    1.0 2012-08-10       635   
...          ...        ...         ...   ...    ...        ...       ...   
155656   -2.3318   104.9000       316.4   1.0    1.0 2019-11-06      1834   
155657   -2.3857   105.0766       328.2   1.1    1.0 2019-11-11       630   
155658   -2.3569   105.0139       327.7   1.1    1.0 2019-11-11       630   
155659   -2.3802   105.1694       318.5   2.1    1.4 2020-09-13       701   
155660   -1.7686   103.9251       324.7   1.2    1.1 2003-06-08       626   

       satellite instrument  confidence  version  bright_t31   frp daynight  \
0          Terra      MODIS          56     6.03       306.0   7.5        D   
1           Aqua      MODIS          57     6.03       302.6  13.9        D   
2           Aqua      MODIS          84     6.03       303.8  31.3        D   
3           Aqua      MODIS          85     6.03       288.4  37.9        D   
4           Aqua      MODIS          57     6.03       295.8  14.5        D   
...          ...        ...         ...      ...         ...   ...      ...   
155656      Aqua      MODIS          93     6.03       289.5  16.0        N   
155657      Aqua      MODIS          69     6.03       299.1  17.9        D   
155658      Aqua      MODIS          81     6.03       303.2  17.0        D   
155659      Aqua      MODIS          63     6.03       281.6  41.3        D   
155660      Aqua      MODIS          78     6.03       302.1  20.4        D   

        type  
0          0  
1          0  
2          0  
3          0  
4          0  
...      ...  
155656     0  
155657     0  
155658     0  
155659     0  
155660     0  

[155661 rows x 15 columns]

### 3. Eksplorasi Datsa Analisis

- agregasi data harian